# Adversarial Search - Connect Four Game
### Assignment 1 - EDAP01 - Artificial Intelligence 
### Author: Hicham Mohamad

## Table of contents
1. [Minimax Search Algorithm with Alph-Beta Pruning](#t1)
2. [Utility Functions and Policy](#t2)
3. [Playing Connect4 Game](#t3)
4. [Main Code](#t4)

## Skeleton

In [1]:
import gym
import random
import requests
import numpy as np

from gym_connect_four import ConnectFourEnv

env: ConnectFourEnv = gym.make("ConnectFour-v0")

#SERVER_ADRESS = "http://localhost:8000/"
SERVER_ADRESS = "https://vilde.cs.lth.se/edap01-4inarow/"
API_KEY = 'nyckel'
STIL_ID = ["hi8826mo-s"] # TODO: fill this list with your stil-id's

In [2]:
def call_server(move):
   res = requests.post(SERVER_ADRESS + "move",
                       data={
                           "stil_id": STIL_ID,
                           "move": move, # -1 signals the system to start a new game. 
                                         # any running game is counted as a loss
                           "api_key": API_KEY,
                       })

   # For safety some respose checking is done here
   if res.status_code != 200:
      print("Server gave a bad response, error code={}".format(res.status_code))
      exit()
   if not res.json()['status']:
      print("Server returned a bad status. Return message: ")
      print(res.json()['msg'])
      exit()
   return res

### Minimax Search Algorithm with Alph-Beta Pruning <a name="t1"/>

**Mini-max adversial search** algorithm is divided in three functions: 
1. the first one is **minimax()** which is the general function acting to get the min or the max value based on the depth it currently explores. 
2. Function **max\_play()** is aimed to maximize the score of the player,  
3. whereas the function **min\_play()** is aimed to minimize the score of opponent. 

**Alpha-beta pruning** is considered as a search algorithm that seeks to decrease the number of nodes that are evaluated by the minimax algorithm in its search tree.
Here we go in the core idea of this algorithm. It maintains two values, $\alpha$ and $\beta$, which represent respectively the **minimum score** that the maximizing player is assured of and the **maximum score** that the minimizing player is assured of. *Initially, $\alpha$ is negative infinity and $\beta$ is positive infinity, i.e. both players start with their worst possible score*. 

Whenever the maximum score that the minimizing player (i.e. the "beta" player) is assured of becomes less than the minimum score that the maximizing player (i.e., the "alpha" player) is assured of (i.e. $\beta \leq \alpha$), the maximizing player need not consider further descendants of this node, as they will never be reached in the actual play.

<img src="alphaBeta2.png">

In [160]:
import copy
#from datetime import datetime

MAX_TIME = 100000

class Optimal_Minimax:

    #def __init__(self, currBoard):
    def __init__(self, env):
        
        #self.time = datetime.now().microsecond
        
        #col = self.minimax(copy.deepcopy(currBoard), -1000000, +1000000, 4)  # XXXX
        new_env = copy.deepcopy(env)
        col = self.minimax(new_env, -1000000, +1000000, 5)  # XXXX
        self.action = col
        
    # mini-max adversial search algorithm is divided in three functions:
    # the first one is minimax which is the general function acting to 
    # get the min or the max value based on the depth it currently explors.
    # return: an action
    #def minimax(self, board, alfa, beta, depth):
    def minimax(self, environ, alfa, beta, depth):
        values_dict = []
        bestValue = -1000000
        #bestMove = -1
        
        legalMoves = environ.available_moves() # XXXXX free columns
        orderedMoves = sorted(legalMoves, key=lambda x: abs(3-x))
                    
        for move in orderedMoves:
            # compare the spent time with the the cut off max time
            #if datetime.now().microsecond - self.time >= MAX_TIME:
            #   return bestMove
                                                
            #next_board = copy.copy(board) # XXXXX
            new_env = copy.deepcopy(environ)
            
            #next_board, reward, done, _ = env.step(move)
            next_board, reward, done, _ = new_env.step(move)
           
            #minValue = self.min_play(next_board, alfa, beta, depth-1) 
            minValue = self.min_play(new_env, alfa, beta, depth-1)  
            values_dict.append((max(bestValue, minValue), move))
            #bestValue = max(bestValue, minValue)
            #if minValue > bestValue:
            #    bestMove = move
            #    bestValue = minValue
            
            #next_board = env.reset(next_board) # XXXXX
        
        bestMove = max(values_dict, key=lambda x: x[0])[1]   

        return bestMove

    # function max_play is aimed to maximize the score of player
    # it only returns the value.
    #def max_play(self, board, alfa, beta, depth):
    def max_play(self, environ, alfa, beta, depth):
        state = environ.board
        legalMoves = environ.available_moves() # XXXX
        
        # if TERMINAL-TEST(state) then return UTILITY(ATATE)
        if (environ.is_win_state()):
            #return countscores(state) + depth # XXXXX
            return streaks_eval(state) + depth # XXXXX
        elif (depth == 0) or (len(legalMoves) == 0):
            #return countscores(state)
            return streaks_eval(state) # XXXXX
        
        # the worst possible score: v <-- -infinity
        bestValue = -1000000   
       
        #for move in legalMoves:
        # ordered ACTIONS
        for a in [3, 2, 4, 1, 5, 0, 6]:
            if a in legalMoves:
                # compare the spent time with the the cut off max time
                #if datetime.now().microsecond - self.time >= MAX_TIME:
                #    return bestValue
                
                #next_board = copy.copy(board) # XXXX
                new_env = copy.deepcopy(environ) # XXXX
            
                next_board, reward, done, _ = new_env.step(a)
            
                #minValue = self.min_play(next_board, alfa, beta, depth-1) 
                minValue = self.min_play(new_env, alfa, beta, depth-1) 
                bestValue = max(bestValue, minValue)
            
                #next_board = env.reset(next_board) # XXXXX
            
                if bestValue >= beta:
                    return bestValue
                alfa = max(alfa, bestValue)

        return bestValue
    
    # function min_play is aimed to minimize the score of opponent. 
    # it only returns the value.
    # the same as Max_play but with roles of alpha, beta reversed
    #def min_play(self, board, alfa, beta, depth):
    def min_play(self, environ, alfa, beta, depth):
        state = environ.board
        legalMoves = environ.available_moves() # XXXX
        
        if (env.is_win_state()):
            #return countscores(state) + depth # XXXXX
            return streaks_eval(state) + depth # XXXXX
        if (depth == 0) or (len(legalMoves) == 0):
            #return countscores(state)
            return streaks_eval(state) # XXXXX
                    
        # initially beta is positive infinity (high)
        # the worst possible score
        bestValue = 1000000
        
        #env.change_player() # change to opponent
              
        # for each a in ACTIONS(state) do
        #for move in legalMoves:
        for a in [3, 2, 4, 1, 5, 0, 6]:
            if a in legalMoves:
                # compare the spent time with the the cut off max time
                #if datetime.now().microsecond - self.time >= MAX_TIME:
                #    return bestValue
                
                #Nex_board = copy.copy(board) # XXXX
                new_env = copy.deepcopy(environ)
            
                new_env.change_player() # change to opponent
           
                next_board, reward, done, _ = new_env.step(a)
           
                new_env.change_player() # change back to student before returning
            
                #maxValue = self.max_play(next_board, alfa, beta, depth-1)
                maxValue = self.max_play(new_env, alfa, beta, depth-1)
                bestValue = min(bestValue, maxValue)
            
                #next_board = env.reset(next_board) # XXXXX
            
            
                if bestValue <= alfa:
                    return bestValue
                beta = min(beta, bestValue)
                        
        return bestValue
    
   

### Heuristic/Utility functions and Policy <a name="t2"/>

In [167]:
def streaks(board, size):
    # Horizontal streaks
    def horizontal(board, size):
        for i in range(env.board_shape[0]):
            for j in range(env.board_shape[1] - size + 1):
                yield board[i][j:j + size]
                

    # iterate over columns on transpose array
    def vertical(board, size):
        return horizontal(board.T, size)
                

    # diagonal streaks
    def diagonal(board, size):
        def diag(board, x, y, size):
            return np.array([board[y+k, x+k] for k in range(size)], dtype=int)
        
        for x in range(env.board_shape[1] - size + 1):
            for y in range(env.board_shape[0] - size + 1):
                yield diag(board, x, y, size)
                    
    def backDiagonal(board, size):
        return diagonal(board[:,::-1], size)
                    

    yield from horizontal(board, size)
    yield from vertical(board, size)
    yield from diagonal(board, size)
    yield from backDiagonal(board, size)

def streaks_eval(board):
    def streakscore(board, size):
        scores = 0
        #print("board iters", len(list(streaks(board, size))))
        
        for square in streaks(board, size):
            if (square == 1).all():
                scores += 1
            elif (square == -1).all():
                scores -= 1
        return scores
    return streakscore(board, 2) + 10000*streakscore(board, 3) + 1000000*streakscore(board, 4)

# return an integer representing the heuristic evaluation score
def col_evaluate(move):
    if move == 3:
        return 10
    if move == 0 or 6:
        return -10
    if move == 1 or 5:
        return -5
    if move == 2 or 4:
        return 5

# return the score of discs own by the player 
# Prefer having your pieces in the center of the board.
#def countscores(self, color):
def countscores(board):
    scores = 0
    scores = longest_streak(board)*10
    
    #for row in range(6):
    #    for col in range(7):
    3        #scores += self.board[row][col]
    #        if (board[row][col] == 1):
    #            scores -= abs(3 - col)
    #        elif (board[row][col] == -1):
    #            scores += abs(3 - col)
                
    #return score if color is BLACK else -score  
    return scores 

# return valid moves with substantial liklihood of winning
def firstavailable_move(board, col, height=6):
    #order = [3,2,4,1,5,0,6]
    #moves = []
    #for col in order:
    for row in range(height):
        if board[row][col] != 0:
            return row - 1
            #print("yess")
            #if env.is_valid_action(i)
                #moves.append([row,col])
                #moves.append(col)
                #break
    #return moves
    return height-1

def max_distance_from(board, x, y) -> int:
        values = []
        
        # (1) Test rows: 
        rvalue = 0
        for j in range(y - 3):
            rvalue = sum(board[x][j:j + 4])
        values.append(rvalue)
        #print("row streaks ", values)

        # (2) Test columns on transpose array
        reversed_board = [list(i) for i in zip(*board)]
        #for i in range(self.board_shape[1]):
        cvalue = 0
        for j in range(x - 3):
            cvalue = sum(reversed_board[x][j:j + 4])
        values.append(cvalue)
        #print("col streaks ", values)

        # (3) Test diagonal
        dvalue = 0
        for i in range(x - 3):
            for j in range(y - 3):
                #dvalue = 0
                for k in range(4):
                    dvalue += board[i + k][j + k]
        values.append(dvalue)
        #print("diag streaks ", values)

        reversed_board = np.fliplr(board)
        # (4) Test reverse diagonal
        ddvalue = 0
        for i in range(x - 3):
            for j in range(y - 3):
                #ddvalue = 0
                for k in range(4):
                    ddvalue += reversed_board[i + k][j + k]
        values.append(ddvalue)
        #print("rev diag streaks ", values)
        
        print("squares values: \n", values)
        print("massimo", max(values))
        return max(values)
    
def longest_streak(board) -> int: 
    longest = 0 
    for i in range(6): 
        for j in range(7): 
            #if board[i][j] == playerid: 
            longest = max( longest, max_distance_from(board, i ,j) ) 
    return longest

#def AI_move(board, AI_color, player_color):
#def AI_move(board):
    #Optimal_Minimax(board)
    #board.print_board()

#### Opponent random agent move

In [168]:
"""
You can make your code work against this simple random agent
before playing against the server.

It returns a move 0-6 or -1 if it could not make a move.
To check your code for better performance, change this code to
use your own algorithm for selecting actions too
"""
def opponents_move(env):
   env.change_player() # change to oppoent
   avmoves = env.available_moves()
    
   if not avmoves:
      env.change_player() # change back to student before returning
      return -1

   # TODO: Optional? XXXXXX TODO xxxxxxx
   # change this to select actions with your policy too
   # that way you get way more interesting games, and you can see 
   # if starting is enough to guarrantee a win
   action = random.choice(list(avmoves))
    
   # the state of the board in "env" is updated 
   # if you play against a local player !!!!!
   state, reward, done, _ = env.step(action)

   if done:
      if reward == 1: # reward is always in current players view
         reward = -1
            
   env.change_player() # change back to student before returning
   return state, reward, done


#### Student move

In [169]:
def student_move(board):
   """
   TODO: Implement your min-max alpha-beta pruning algorithm here.
   Give it whatever input arguments you think are necessary
   (and change where it is called).
   The function should return a move from 0-6
   """
   #return random.choice([0, 1, 2, 3, 4, 5, 6])
   #AI_move(board, AI_color, player_color)
   #AI_move(board)
   alfaBeta = Optimal_Minimax(env)
   studentMove = alfaBeta.action 
   #print("student_move", studentMove) 
   #env.change_player() # change back to opponent before returning
   return studentMove 
    

### Playing Connect4 game <a name="t3"/>

In [170]:
def play_game(vs_server = False):
   """
   The reward for a game is as follows. You get a
   botaction = random.choice(list(avmoves)) reward from the
   server after each move, but it is 0 while the game is running
   loss = -1
   win = +1
   draw = +0.5
   error = -10 (you get this if you try to play in a full column)
   Currently the player always makes the first move
   """

   # default state
   state = np.zeros((6, 7), dtype=int)

   # setup new game
   if vs_server:
      # Start a new game by calling the server !!!!!
      res = call_server(-1) # -1 signals the system to start a new game. 
                            # any running game is counted as a loss

      # This should tell you if you or the bot starts
      print(res.json()['msg'])
        
      # get the bot move and the state
      # But it does not update the board in "env" with that. !!!!
      # Unless you decide to do that - which I think is what I would do
      #  You can set your env's state to that using env.reset(state)
      botmove = res.json()['botmove']
      state = np.array(res.json()['state'])
    
   else:
      # reset game to starting state
      env.reset(board=None)
      # determine first player
      student_gets_move = random.choice([True, False])
      if student_gets_move:
         print('You start!')
         print()
      else:
         print('Bot starts!')
         print()

   # Print current game state
   print("Current state (1 are student discs, -1 are servers, 0 is empty): ")
   print(state)
   print()

   done = False
   while not done:
      # Select your move
      stmove = student_move(state) # TODO: change input here XXXXX TO DO TO DO XXXXXX
      #print("main student move: ", stmove)

      # make both student and bot/server moves
      if vs_server:
         # Send your move to server and get response
         res = call_server(stmove)
         print(res.json()['msg'])

         # Extract response values from the server
         # Nota Bene: When receiving the state from the server, 
         # one can decide how to go with it, there are several ways: 
         # one can use the "env" object and reset()it, 
         # or use the "step()" function, or write its own class for it. 
         result = res.json()['result']
         botmove = res.json()['botmove']
         state = np.array(res.json()['state'])
         state = env.reset(state) # XXXXXXXXXXXXXX TO DO XXXXXXXXXXX !!!!!!!
      else:
         if student_gets_move:
            # Execute your move
            avmoves = env.available_moves()
            if stmove not in avmoves:
               print("You tried to make an illegal move! Games ends.")
               break
            state, result, done, _ = env.step(stmove)

         student_gets_move = True # student only skips move first turn if bot starts

         # print or render state here if you like

         # select and make a move for the opponent, 
         # returned reward from students view
         if not done:
            state, result, done = opponents_move(env)

      # Check if the game is over
      if result != 0:
         done = True
         if not vs_server:
            print("Game over. ", end="")
         if result == 1:
            print("You won!")
         elif result == 0.5:
            print("It's a draw!")
         elif result == -1:
            print("You lost!")
         elif result == -10:
            print("You made an illegal move and have lost!")
         else:
            print("Unexpected result result={}".format(result))
         if not vs_server:
            print("Final state (1 are student discs, -1 are servers, 0 is empty): ")
         
         # necessary for getting results of a streak of 20 games 
         if result == 1:
             return 1
         else:
             return 0
        
      else:
         print("Current state (1 are student discs, -1 are servers, 0 is empty): ")

      # Print current gamestate
      print(state)
      print()

### Main code <a name="t4"/>

In [172]:
def main():
   #play_game(vs_server = True)
   
   wins = 0 
   gamesNbr = 60
   for game in range(gamesNbr):
       wins += play_game(vs_server = True)
   print('{}/{}'.format(wins, gamesNbr))
   input()
   
   # TODO: Change vs_server to True when you are ready 
   #       to play against the server
   # the results of your games there will be logged

if __name__ == "__main__":
    main()

New game initiated, you start. Make your move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0 -1  1  0  0  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0 -1  1  0  0  0]
 [ 0  0 -1  1  0  0  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0]
 [ 0  0 -1  1  0  0  0]
 [ 0  0 -1  1  0  0 -1]]

The server has made its move.
Current state (1 are studen

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  1 -1  0  0  0]
 [-1 -1  1  1 -1 -1  0]
 [ 1  1  1 -1  1  1  0]
 [-1  1 -1  1 -1 -1 -1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  1 -1  0  1  0]
 [-1 -1  1  1 -1 -1  0]
 [ 1  1  1 -1  1  1  0]
 [-1  1 -1  1 -1 -1 -1]]

The game has ended.
You won!
New game initiated, you start. Make your move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0 -1  0  1  0  0]]

The server has made its move

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0  1  0 -1  0]
 [ 0 -1 -1  1  0 -1  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  1  0]
 [ 0 -1  0  1  0 -1  0]
 [ 0 -1 -1  1  0 -1  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  1  0  1  0  1  0]
 [ 0 -1  0  1  0 -1  0]
 [ 0 -1 -1  1 -1 -1  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0 -1  0 -1  0  0  0]
 [ 0  1  0  1  0  1  0]
 [ 0 -1  1  1  0 -1  0]
 [ 0 -1 -1  1 -1 -1  0]]

The server has made its move.
Cu

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0 -1  0  1  0 -1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0 -1 -1  1  0 -1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0 -1 -1  1  0 -1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0  0  1  1  0  0]
 [-1  0 -1 -1  1  0 -1]]

The server has made its move.
Cu

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1 -1]
 [ 0 -1 -1  0  0  1  1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1 -1]
 [ 0 -1 -1  1 -1  1  1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0 -1  1  0  1 -1]
 [ 0 -1 -1  1 -1  1  1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0 -1  0]
 [ 0  0 -1  1  0  1 -1]
 [ 0 -1 -1  1 -1  1  1]]

The game has ended.
You won!
New

...

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  1 -1  0 -1  0  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0]
 [ 0  1 -1  0 -1  0 -1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0]
 [ 0  1  0  0  0  0  0]
 [ 0  1 -1  0 -1 -1 -1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0]
 [ 0  1  0  0 -1  0  0]
 [ 0  1 -1  1 -1 -1 -1]]

The game has ended.
You won!
New

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  1  1  0  0  0]
 [ 0  0 -1  1  0  0  0]
 [ 0  0 -1  1 -1  0 -1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  1  1  0  0  0]
 [ 0  0 -1  1  0  0  0]
 [ 0  0 -1  1 -1 -1 -1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  1  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  1 -1  0  0  0]
 [ 0  0  1  1  0  0  0]
 [ 0  0 -1  1  0 -1  0]
 [ 0  0 -1  1 -1 -1 -1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  1  0  0  0]
 [ 0  0 -1  1  0  0  0]
 [ 0  0  1 -1  0  0  0]
 [ 0  0  1  1  0  1  0]
 [ 0  0 -1  1  0 -1  0]
 [ 0  0 -1  1 -1 -1 -1]]

The server has made its move.
Cu

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0]
 [ 0  0  1  0  0  0  0]
 [ 0  0  1  1  0 -1  0]
 [ 0  0  1 -1  0 -1  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0]
 [ 0  0 -1  0  0  0  0]
 [ 0  0  1  0  0  1  0]
 [ 0  0  1  1  0 -1  0]
 [ 0  0  1 -1  0 -1  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0]
 [ 0  0 -1  0  0  0  0]
 [ 0  0  1  1  0  1  0]
 [ 0  0  1  1  0 -1  0]
 [-1  0  1 -1  0 -1  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0]
 [ 0  0 -1  1  0  0  0]
 [ 0  0  1  1  0  1  0]
 [ 0  0  1  1  0 -1  0]
 [-1  0  1 -1  0 -1 -1]]

The game has ended.
You won!
New

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0 -1  1  0  0  0]
 [ 0 -1  1  1  0  0 -1]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0 -1  1  0  0 -1]
 [ 0 -1  1  1  0  0 -1]]

The game has ended.
You won!
New game initiated, you start. Make your move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0]]

The server has made its move

The game has ended.
You won!
New game initiated, you start. Make your move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0]
 [ 0  0  1  0  0  0  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0]
 [ 0 -1  1  1  0  0  0]]

The server has made its move.
Current state (1 are student discs, -1 are servers, 0 is empty): 
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0 -1 -1  1  0  0  0]
 [ 0 -1  1  1  0  0  0]]

The server has made its move